In [ ]:
library(Seurat)
library(openxlsx)
library(readxl)
library(Matrix)
library(readr)


Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.3.2 but the current version is
4.3.3; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect




In [13]:
#Reyf
setwd("/dh-projects/ag-conrad/analysis/saal11/panfib/Renaming/Reyf")
#Upload matrices
#data_reyf_matrix <- readMM("data_reyf_matrix.mtx")
#Upload final gene list
data_reyf_genes <- read.xlsx("data_reyf_geneIDs_int_removed_manualedit.xlsx")
data_reyf_genes <- data_reyf_genes$New
data_reyf_cellIDs <- read.delim("data_reyf_cellIDs.tsv")
rownames(data_reyf_matrix) <- data_reyf_genes
colnames(data_reyf_matrix) <- data_reyf_cellIDs$cellIDs

In [ ]:
rownames(data_reyf_matrix)

In [14]:
#Find sums of duplicate genes (from manual edit)
duplicate_genes <- duplicated(rownames(data_reyf_matrix))
cat("Duplicate gene IDs:\n")
print(rownames(data_reyf_matrix)[duplicate_genes])
duplicated <- rownames(data_reyf_matrix)[duplicate_genes]

Duplicate gene IDs:
 [1] "BTBD8"        "HNRNPU"       "RRM2"         "MCM2"         "ROPN1L"      
 [6] "GCNT4"        "TRAF3IP2-AS1" "LINC00632"    "TMEM8B"       "S1PR3"       
[11] "MICAL2"       "NEBL"         "LCOR"         "SUCLA2"       "MIA2"        
[16] "MYO18A"       "IGFLR1"       "C10orf67"     "PLEKHG7"      "DLGAP2"      
[21] "MEG8"         "MEG8"        


In [27]:
#Find the zeros of these duplicates
#Initialize an empty list to store the sums
duplicate_genes <- read.xlsx("data_reyf_geneIDs_old_dup.xlsx")
sums_list <- list()

#Iterate over each gene in your list
for (gene in duplicated) {
  #Subset the count matrix for the current gene and calculate the sum
  gene_sum <- sum(data_reyf_matrix[rownames(data_reyf_matrix) == gene, ])
  
  #Add the sum to the sums_list with the gene name as the key
  sums_list[[gene]] <- gene_sum
}

In [ ]:
sums_list

In [16]:
#Filter zeros
row_sums <- rowSums(data_reyf_matrix)
names_with_zeros <- names(row_sums[row_sums == 0])
names_with_zeros

[1] "SCGN"    "RBMXL2"  "DELEC1"  "TCERG1L"

In [17]:
filtered_data_reyf_matrix <- data_reyf_matrix[row_sums != 0, ]

In [18]:
#Concatenating replicates
filtered_data_reyf_matrix <- as(filtered_data_reyf_matrix, "dgCMatrix")

'as(<dgTMatrix>, "dgCMatrix")' is deprecated.
Use 'as(., "CsparseMatrix")' instead.
See help("Deprecated") and help("Matrix-deprecated").



In [19]:
gene_names <- rownames(filtered_data_reyf_matrix)
unique_gene_names <- unique(gene_names)
gene_factors <- factor(gene_names, levels = unique_gene_names)

In [20]:
aggregated_counts <- rowsum(as.matrix(filtered_data_reyf_matrix), group = gene_factors)

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 9.9 GiB”


In [21]:
str(aggregated_counts)

 num [1:22901, 1:57695] 0 0 0 0 0 0 0 0 1 0 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:22901] "RP11-34P13.7" "FO538757.2" "AP006222.2" "RP4-669L17.10" ...
  ..$ : NULL


In [22]:
str(filtered_data_reyf_matrix)

Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:103111994] 8 36 38 53 56 59 66 67 69 74 ...
  ..@ p       : int [1:57696] 0 2034 6192 8394 10689 11989 14804 18093 19909 21944 ...
  ..@ Dim     : int [1:2] 22923 57695
  ..@ Dimnames:List of 2
  .. ..$ : chr [1:22923] "RP11-34P13.7" "FO538757.2" "AP006222.2" "RP4-669L17.10" ...
  .. ..$ : NULL
  ..@ x       : num [1:103111994] 1 2 1 1 3 1 2 1 1 1 ...
  ..@ factors : list()


In [ ]:
#Dimensions make sense?
#Yes the difference matches the number of duplicates

In [24]:
#Creating a seurat object
data_reyf_metadata <- read_csv("data_reyf_metadata_final.csv")

New names:
• `` -> `...1`
Rows: 57695 Columns: 17
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (11): ...1, lineage, condition, study, tissue, primary_or_cultured, cell...
dbl  (4): nCount_RNA, nFeature_RNA, percent.mt, age
lgl  (2): subclass, aetiology

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [27]:
data_reyf_metadata <- as.data.frame(data_reyf_metadata)

In [29]:
rownames(data_reyf_metadata) <- data_reyf_metadata$`...1`

In [30]:
data_reyf_metadata$`...1` <- NULL

In [33]:
data_reyf_seurat <- CreateSeuratObject(counts = aggregated_counts, meta.data = data_reyf_metadata)

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”


In [35]:
save(data_reyf_seurat, file = "data_reyf_seurat_final.RData")